<a href="https://colab.research.google.com/github/fanaev/tadimo-labs/blob/main/blank__05_NLP_1_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [155]:
import re
import nltk

import torch
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn as nn
import torch.optim as optim
import torchtext

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder

from tqdm import tqdm

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Train func

In [3]:
def train_model(_model, dataloaders, criterion, 
                optimizer, num_epochs = 10, acc_stop_value = 0.9,
                num_classes = 2, lr = 1e-3):
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  _model = _model.to(device)

  train_losses, test_losses = [], []

  _trainloader = dataloaders[0]
  _valloader = dataloaders[1]
  for e in range(epochs):
    model.train()
    running_loss = 0
    print(f'Epo: {e}')
    print('Train step')
    for X, y in tqdm(trainloader):
      X, y = X.to(device), y.to(device)
      optimizer.zero_grad()
      
      output = model(X)
      loss = criterion(output, y)
      loss.backward()
      optimizer.step()
      
      running_loss += loss.item()
    else:
      test_loss = 0
      accuracy = 0
      
    with torch.no_grad():
        model.eval()
        print('Test step')
        for X, y in tqdm(testloader):
          X, y = X.to(device), y.to(device)
          log_ps = model(X)
          test_loss += criterion(log_ps, y)
          
          ps = torch.exp(log_ps)
          top_p, top_class = ps.topk(1, dim = 1)
          equals = top_class == y.view(*top_class.shape)
          accuracy += torch.mean(equals.type(torch.FloatTensor))
      
        train_losses.append(running_loss/len(trainloader))
        test_losses.append(test_loss/len(testloader))
        print('---------------------')
        print("Training loss: {:.3f}..".format(running_loss/len(trainloader)),
            "Test loss: {:.3f}..".format(test_loss/len(testloader)),
            "Test Accuracy: {:.3f}".format(accuracy/len(testloader)))    

## 1. Представление и предобработка текстовых данных 

1.1 Операции по предобработке:
* токенизация
* стемминг / лемматизация
* удаление стоп-слов
* удаление пунктуации
* приведение к нижнему регистру
* любые другие операции над текстом

In [4]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.snowball import SnowballStemmer

In [5]:
text = 'Select your preferences and run the install command. Stable represents the most currently tested and supported version of PyTorch. Note that LibTorch is only available for C++'

Реализовать функцию `preprocess_text(text: str)`, которая:
* приводит строку к нижнему регистру
* заменяет все символы, кроме a-z, A-Z и знаков .,!? на пробел


In [120]:
def preprocess_text(text: str):
  text = text.lower()
  pattern = r'[^a-zA-Z0-9.,!?\s]'

  text = re.sub(string = text, pattern = pattern, repl = '')
  return text

In [121]:
preprocess_text(text)

'select your preferences and run the install command. stable represents the most currently tested and supported version of pytorch. note that libtorch is only available for c'

In [99]:
re.sub(pattern =  r'\s+', string = 'hi   susa   ', repl = '')

'hisusa'

1.2 Представление текстовых данных при помощи бинарного кодирования


Представить первое предложение из `text` в виде тензора `sentence_t`: `sentence_t[i] == 1`, если __слово__ с индексом `i` присуствует в предложении.

In [ ]:
# Это задание реализовано в "Классификация обзоров ресторанов"

## 2. Классификация фамилий по национальности

Датасет: https://disk.yandex.ru/d/owHew8hzPc7X9Q?w=1

2.1 Считать файл `surnames/surnames.csv`. 

2.2 Закодировать национальности числами, начиная с 0.

2.3 Разбить датасет на обучающую и тестовую выборку

2.4 Реализовать класс `Vocab` (токен = __символ__)

2.5 Реализовать класс `SurnamesDataset`

2.6. Обучить классификатор.

2.7 Измерить точность на тестовой выборке. Проверить работоспособность модели: прогнать несколько фамилий студентов группы через модели и проверить результат. Для каждой фамилии выводить 3 наиболее вероятных предсказания.

### Подготовка данных

In [191]:
import zipfile
from tqdm import tqdm

zf = zipfile.ZipFile('/content/surnames.zip')
for file in tqdm(zf.infolist()):
    zf.extract(file)

100%|██████████| 2/2 [00:00<00:00, 623.50it/s]


In [192]:
df_surnames = pd.read_csv('/content/surnames/surnames.csv')

In [207]:
class Vocab:
  def __init__(self, data):
    data = data.surname.unique().astype('str')
    self.idx_to_token = {}
    self.token_to_idx = {}
    self.gennerate_vocab(data)
    self.vocab_len = len(self.token_to_idx)
  def add_token(self, token):
    if token not in self.token_to_idx:
      self.token_to_idx[token] = len(self.token_to_idx)
      self.idx_to_token[len(self.idx_to_token)] = token

  def gennerate_vocab(self, data):
    for surname in data:
      for char in preprocess_text(surname):
        self.add_token(char)

nationalities = df_surnames.nationality.unique()
vocab_nat = {nationalities[i]: i for i in range(len(nationalities))}
reverse_nat =  {i: nationalities[i] for i in range(len(nationalities))}
vocab = Vocab(df_surnames)

In [194]:
vocab.vocab_len

27

In [195]:
class SurnamesDataset(Dataset):
  def __init__(self, X, y, vocab: Vocab):
    self.X = X
    self.y = y
    self.vocab = vocab

  def vectorize(self, surname):
    '''Генерирует представление фамилии surname в при помощи бинарного кодирования (см. 1.2)'''
    repr = torch.zeros(self.vocab.vocab_len)
    for char in preprocess_text(surname):
      idx = self.vocab.token_to_idx[char]
      repr[idx] = 1
    return repr
    
  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    return self.vectorize(self.X[idx]), vocab_nat[self.y[idx]]

In [196]:
dataset = SurnamesDataset(X = df_surnames.surname, y = df_surnames.nationality, vocab = vocab)

train_size = int(dataset.__len__()*0.8)
test_size = dataset.__len__() - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

trainloader = torch.utils.data.DataLoader(train_dataset, batch_size = 32, shuffle = True, pin_memory=True)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size = 1, pin_memory = True)

### Обучение MLP

In [197]:
model = nn.Sequential(
    nn.LazyLinear(512),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.LazyLinear(1024),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.LazyLinear(len(vocab_nat)),
)
optimizer = torch.optim.Adam(model.parameters(),lr=1e-3)
criterion = nn.CrossEntropyLoss()
epochs = 10

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [198]:
train_model(_model = model,
              dataloaders = [trainloader, trainloader],
              criterion = criterion,
              optimizer = optimizer
              )

Epo: 0
Train step


100%|██████████| 275/275 [00:00<00:00, 293.10it/s]


Test step


100%|██████████| 2196/2196 [00:01<00:00, 1576.45it/s]


---------------------
Training loss: 1.662.. Test loss: 1.457.. Test Accuracy: 0.566
Epo: 1
Train step


...

---------------------
Training loss: 0.957.. Test loss: 1.232.. Test Accuracy: 0.633
Epo: 9
Train step


100%|██████████| 275/275 [00:00<00:00, 290.77it/s]


Test step


100%|██████████| 2196/2196 [00:01<00:00, 1608.88it/s]

---------------------
Training loss: 0.911.. Test loss: 1.235.. Test Accuracy: 0.637


### Eval

In [211]:
surnames = ['Topson', 'Xio', 'Ivanov']
with torch.no_grad():
  for surname in surnames:
    log_ps = model(dataset.vectorize(surname).to('cuda:0'))
    ps = torch.exp(log_ps)
    nationality = reverse_nat[ps.topk(1)[1].item()]
    print('Предсказанная национальдность для фамилии ' + surname + ':', 
          nationality
          )

Предсказанная национальдность для фамилии Topson: English
Предсказанная национальдность для фамилии Xio: Chinese
Предсказанная национальдность для фамилии Ivanov: Russian


## 3. Классификация обзоров ресторанов

Датасет: https://disk.yandex.ru/d/nY1o70JtAuYa8g

3.1 Считать файл `yelp/raw_train.csv`. Оставить от исходного датасета 10% строчек.

3.2 Воспользоваться функцией `preprocess_text` из 1.1 для обработки текста отзыва. Закодировать рейтинг числами, начиная с 0.

3.3 Разбить датасет на обучающую и тестовую выборку

3.4 Реализовать класс `Vocab` (токен = слово)

3.5 Реализовать класс `ReviewDataset`

3.6 Обучить классификатор

3.7 Измерить точность на тестовой выборке. Проверить работоспособность модели: придумать небольшой отзыв, прогнать его через модель и вывести номер предсказанного класса (сделать это для явно позитивного и явно негативного отзыва)


### Подготовка данных

In [122]:
class Vocab:
  def __init__(self, data):
    self.data = data
    self.idx_to_token = {}
    self.token_to_idx = {}
    self.gennerate_vocab()
    self.vocab_len = len(self.token_to_idx)
  def add_token(self, token):
    if token not in self.token_to_idx:
      self.token_to_idx[token] = len(self.token_to_idx)
      self.idx_to_token[len(self.idx_to_token)] = token

  def gennerate_vocab(self):
    for row in self.data:
      for word in row.split(' '):
        self.add_token(word)

In [146]:
class ReviewDataset(Dataset):
  def __init__(self, X, y, vocab: Vocab):
    self.X = X
    self.y = y
    self.vocab = vocab

  def vectorize(self, review):
    '''Генерирует представление фамилии surname в при помощи бинарного кодирования (см. 1.2)'''
    repr = torch.zeros(self.vocab.vocab_len)
    for word in review.split(' '):
      idx = self.vocab.token_to_idx[word]
      repr[idx] = 1
    return repr
    
  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    return self.vectorize(self.X[idx]), self.y[idx]

In [147]:
traindata = pd.read_csv(
    '/content/raw_train.csv', 
    sep = '","', 
    error_bad_lines=False, 
    header = None
    ).sample(frac = 0.1)

X = traindata[[1]].apply(
    lambda x: preprocess_text(x.iloc[0]), 
    axis = 1
    ).values
labels = traindata[[0]].apply(
    lambda x: int(x.iloc[0].replace('"', '')), 
    axis = 1
    ).values - 1 # [1, 2] -> [0, 1]
word_vocab = Vocab(X)

del traindata

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
Skipping line 23513: Expected 2 fields in line 23513, saw 3. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.
Skipping line 126780: Expected 2 fields in line 126780, saw 3. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.


In [148]:
dataset = ReviewDataset(X = X, y = labels, vocab = word_vocab)

train_size = int(dataset.__len__()*0.8)
test_size = dataset.__len__() - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

trainloader = torch.utils.data.DataLoader(train_dataset, batch_size = 32, shuffle = True, pin_memory=True)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size = 1, pin_memory = True)

In [149]:
train_dataset.__getitem__(0)

(tensor([1., 1., 0.,  ..., 0., 0., 0.]), 1)

### Обучение MLP

In [159]:
model = nn.Sequential(
    nn.LazyLinear(512),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.LazyLinear(1024),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.LazyLinear(2),
)
optimizer = torch.optim.Adam(model.parameters(),lr=1e-4)
criterion = nn.CrossEntropyLoss()
epochs = 1

In [160]:
train_model(_model = model,
              dataloaders = [trainloader, trainloader],
              criterion = criterion,
              optimizer = optimizer
              )

Epo: 0
Train step


100%|██████████| 1400/1400 [02:04<00:00, 11.24it/s]


Test step


100%|██████████| 11200/11200 [00:36<00:00, 311.09it/s]

---------------------
Training loss: 0.270.. Test loss: 0.214.. Test Accuracy: 0.913


### Eval

In [173]:
pos = 'like very much'

In [174]:
neg = 'so bad'

In [190]:
with torch.no_grad():
  print("Предсказание для позивного примера:",
      torch.exp(model(dataset.vectorize(pos).to('cuda:0'))).topk(1)[1].item()
      )
  print("Предсказание для негативного примера:",
      torch.exp(model(dataset.vectorize(neg).to('cuda:0'))).topk(1)[1].item()
      )

Предсказание для позивного примера: 1
Предсказание для негативного примера: 0
